In [4]:
import sys
import json
from pprint import pprint
from src.api.six import *

def print_section(title):
    print("\n" + "="*50)
    print(title)
    print("="*50 + "\n")

In [17]:
print_section("1. Testing query_llm")
test_query = "What are the main industries in Switzerland?"
llm_response = query_llm(test_query)
llm_response["content"]


1. Testing query_llm



"Switzerland has a diverse economy with several key industries that contribute significantly to its GDP. The main industries in Switzerland include:\n\n1. **Financial Services**: Switzerland is known for its robust banking and financial services sector, including private banking, asset management, and insurance.\n\n2. **Pharmaceuticals and Biotechnology**: The country is home to major pharmaceutical companies like Novartis and Roche, making this sector a significant contributor to the economy.\n\n3. **Manufacturing**: Switzerland has a strong manufacturing sector, particularly in precision machinery, electronics, and watchmaking.\n\n4. **Tourism**: The picturesque landscapes and cultural heritage make tourism a vital industry, attracting millions of visitors each year.\n\n5. **Information Technology**: The IT sector is growing rapidly, with many companies focusing on software development, cybersecurity, and digital services.\n\n6. **Agriculture**: Although it represents a smaller porti

In [88]:
# 2. Testing query_agent
print_section("2. Testing query_agent")
agent_query = "Get me NVIDIAS stock price for 2020"
agent_response = query_agent(agent_query)
pprint(agent_response)



2. Testing query_agent

{'messages': [{'additional_kwargs': {},
               'content': 'Get me NVIDIAS stock price for 2020',
               'example': False,
               'id': 'fd9e29b7-0827-49f4-8729-50b1982cadde',
               'name': None,
               'response_metadata': {},
               'type': 'human'},
              {'additional_kwargs': {'refusal': None,
                                     'tool_calls': [{'function': {'arguments': '{"query":"NVIDIA","first":"01.01.2020","last":"31.12.2020"}',
                                                                  'name': 'Historical_Price_Data'},
                                                     'id': 'call_pxydmJBynSTI5tb3TZQVSCC4',
                                                     'type': 'function'}]},
               'content': '',
               'example': False,
               'id': 'run-b22d824d-a182-4fdd-8886-fd725a02844c-0',
               'invalid_tool_calls': [],
               'name': None,
          

In [89]:
agent_response

{'messages': [{'content': 'Get me NVIDIAS stock price for 2020',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': 'fd9e29b7-0827-49f4-8729-50b1982cadde',
   'example': False},
  {'content': '',
   'additional_kwargs': {'tool_calls': [{'id': 'call_pxydmJBynSTI5tb3TZQVSCC4',
      'function': {'arguments': '{"query":"NVIDIA","first":"01.01.2020","last":"31.12.2020"}',
       'name': 'Historical_Price_Data'},
      'type': 'function'}],
    'refusal': None},
   'response_metadata': {'token_usage': {'completion_tokens': 37,
     'prompt_tokens': 1609,
     'total_tokens': 1646,
     'completion_tokens_details': {'accepted_prediction_tokens': 0,
      'audio_tokens': 0,
      'reasoning_tokens': 0,
      'rejected_prediction_tokens': 0},
     'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
    'model_name': 'gpt-4o-mini-2024-07-18-2024-07-18',
    'system_fingerprint': 'fp_ded0d14823',
    'prompt_filter_results': [{'pr

In [90]:
import pandas as pd


message = agent_response['messages'][2]
item = message.get("item")
if item is None:
    raise NotImplementedError(f"Agent did not return data for the query. Received answer: {agent_response['content']}.")

item = json.loads(item)
tool = item["tool"]

# If it is OHLC
if tool.lower() == "ohlc":
    data = json.loads(item["data"])
    if len(data) != 1:
        raise NotImplementedError("There seem to be multiple time series in the ohlc data. Implement handling.")

    stock_handle, time_series_str = next(iter(data.items()))
    time_series = json.loads(time_series_str)
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)

    rechart_data = [
        {
            "name": date.strftime("%Y-%m-%d"),
            "open": row["open"],
            "high": row["high"],
            "low": row["low"],
            "close": row["close"],
            "volume": row["vol"]
        }
        for date, row in df.iterrows()
    ]




In [91]:
rechart_data

[{'name': '2020-01-02',
  'open': 5.97125,
  'high': 5.99775,
  'low': 5.91875,
  'close': 5.99775,
  'volume': 56359840},
 {'name': '2020-01-03',
  'open': 5.875,
  'high': 5.94375,
  'low': 5.853,
  'close': 5.90175,
  'volume': 53267760},
 {'name': '2020-01-06',
  'open': 5.809,
  'high': 5.93125,
  'low': 5.78275,
  'close': 5.9265,
  'volume': 77251600},
 {'name': '2020-01-07',
  'open': 5.955,
  'high': 6.044,
  'low': 5.90975,
  'close': 5.99825,
  'volume': 72582920},
 {'name': '2020-01-08',
  'open': 5.994,
  'high': 6.0505,
  'low': 5.954,
  'close': 6.0095,
  'volume': 68742080},
 {'name': '2020-01-09',
  'open': 6.08875,
  'high': 6.1475,
  'low': 6.02275,
  'close': 6.0755,
  'volume': 60848360},
 {'name': '2020-01-10',
  'open': 6.187,
  'high': 6.2125,
  'low': 6.09375,
  'close': 6.108,
  'volume': 63137760},
 {'name': '2020-01-13',
  'open': 6.1915,
  'high': 6.3245,
  'low': 6.16875,
  'close': 6.2995,
  'volume': 83393920},
 {'name': '2020-01-14',
  'open': 6.26275,


In [5]:

# 3. Testing searchwithcriteria
print_section("3. Testing searchwithcriteria")
criteria_query = '{"ebitda": "is positive", "employees": "more than 10000"}'
criteria_response = searchwithcriteria(criteria_query)
pprint(criteria_response)


3. Testing searchwithcriteria

{'message': 'The most relevant data for your query is:\n'
            '**The following table has already been shown to the user, do not '
            'show it again.**\n'
            '|    | Name                | SIX_ID            | ISIN         |   '
            'Valornumber (listing) | Bourse '
            'Code                                        | '
            'Currency                       |   Fundamentals annual 1 - EBITDA '
            '(millions) |   Fundamentals annual 1 - Employees (millions) |\n'
            '|---:|:--------------------|:------------------|:-------------|------------------------:|:---------------------------------------------------|:-------------------------------|--------------------------------------------:|-----------------------------------------------:|\n'
            '|  0 | NVIDIA Rg           | 994529,67,333     | US67066G1040 '
            '|                  994529 | NMS - "NASDAQ" '
            '(67)           

In [92]:
# 4. Testing ohlcv
print_section("4. Testing ohlcv")
# Test with just start date
ohlcv_response1 = ohlcv("banco santander", first="01.01.2024")
pprint(ohlcv_response1)


4. Testing ohlcv

{'message': 'Plots correctly shown to user. Answer the user.\n'
            '\n'
            '|                    |   First |   Last |   Min |   Max | '
            'Return   |\n'
            '|:-------------------|--------:|-------:|------:|------:|:---------|\n'
            '| Banco Santander Rg |    3.79 |  6.561 | 3.563 | 6.595 | '
            '53.22%   |',
 'object': '{"tool": "OHLC", "data": "{\\"Banco Santander Rg\\": '
           '\\"{\\\\\\"2024-01-02T00:00:00.000\\\\\\":{\\\\\\"open\\\\\\":3.79,\\\\\\"high\\\\\\":3.89,\\\\\\"low\\\\\\":3.7895,\\\\\\"close\\\\\\":3.8535,\\\\\\"vol\\\\\\":22855335,\\\\\\"Total '
           'return\\\\\\":\\\\\\"0.00%\\\\\\",\\\\\\"Anualized '
           'return\\\\\\":\\\\\\"0.00%\\\\\\",\\\\\\"Max\\\\\\":3.89,\\\\\\"Min\\\\\\":3.7895},\\\\\\"2024-01-03T00:00:00.000\\\\\\":{\\\\\\"open\\\\\\":3.87,\\\\\\"high\\\\\\":3.88,\\\\\\"low\\\\\\":3.789,\\\\\\"close\\\\\\":3.7995,\\\\\\"vol\\\\\\":23133126,\\\\\\"Total '
           '

In [93]:
# 5. Testing companydatasearch
print_section("5. Testing companydatasearch")
company_query = '{"Caixabank": "number of employees|2023", "BBVA": "number of employees|2023"}'
company_response = companydatasearch(company_query)
pprint(company_response)


5. Testing companydatasearch

{'message': '|    | Name      | ISIN         | '
            'BC                                                              | '
            'Fundamentals 2024 - Employees (millions)   | Fundamentals 2021 - '
            'Employees (millions)   | Fundamentals 2020 - Employees '
            '(millions)   | Fundamentals 2023 - Employees (millions)   | '
            'Fundamentals 2022 - Employees (millions)   |\n'
            '|---:|:----------|:-------------|:----------------------------------------------------------------|:-------------------------------------------|:-------------------------------------------|:-------------------------------------------|:-------------------------------------------|:-------------------------------------------|\n'
            '|  0 | Caixabank | ES0140609019 | STN - "BME Bolsas y Mercados '
            'Espanoles - Renta Variable" (1058) | '
            "45'789                                     | "
            "49'634   

In [94]:
# 6. Testing summary
print_section("6. Testing summary")
summary_response = summary("UBS")
pprint(summary_response)


6. Testing summary

{'message': 'The Urbas Grupo Fin. Br of UBS is Valor '
            'number                                                         '
            '468959\n'
            'Ticker '
            'symbol                                                           '
            'UBS\n'
            'ISIN                                                           '
            'ES0182280018\n'
            'Instrument type           1 - "Share, unit, particip. cert. in '
            'companies...\n'
            'Outstanding Securities                                          '
            '15962047650\n'
            'Outstanding Capital                                             '
            '159620476.5\n'
            'Dividend policy                               "Other payment '
            'frequency" (9)\n'
            'Name                                                    Urbas '
            'Grupo Fin. Br\n'
            'open                                          